<img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logo.png" width="40%"  align="left">

# Creating constant step-size mass-balance profiles via interpolation for OGGM 

In this notebook, we use the created selected link file together with the raw MB profile data to create mass-balance profiles with a constant step size (i.e. altitude difference for all profile levels, here: dh = 50m) via interpolation. 
- Those MB profiles of a year for glaciers with just one measurement are removed as we can not create a profile with just a single measurement. 
- in addition, two glaciers had single measurements for some years with very large differences between Upper and Lower BOUNDARY (>150m) while they had other measurements with lower differences in that MB elevation altitudinal band. These measurements were discarded for the MB profiles!

This notebook was initially created by Moritz Oberrauch and was updated and modified by Lilian Schuster.

### Import section

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

### Read the WGMS files

Change paths to necesary folders and data files

In [ ]:
# just download the newest data, change the path_to_download_data and the year and month accordingly. If you run the entire notebook, the new WGMS MB data should be processed for OGGM
year = '2021'
month = '05'
path_to_download_data = '/home/lilianschuster/Downloads/'

In [ ]:
# working directory
idir = f'{path_to_download_data}DOI-WGMS-FoG-{year}-{month}'
# WMGS lookup table
#df_links = pd.read_csv(os.path.join(idir, f'WGMS-FoG-{year}-{month}-AA-GLACIER-ID-LUT.csv'), encoding='iso8859_15')
# WGMS FoG data file
df_mb_all = pd.read_csv(os.path.join(idir, f'WGMS-FoG-{year}-{month}-EE-MASS-BALANCE.csv'), encoding='iso8859_15')

In [ ]:
# output directory
odir = '/home/lilianschuster/oggm-sample-data/wgms/'
#odir = '/Users/oberrauch/Documents/Studium/work/WGMS_Profiles'
df_links_sel = pd.read_csv(os.path.join(odir, 'rgi_wgms_links_20220112.csv'))

## Statistics about the height levels
In order to choose one standardized step size for all mb profile, we'll take a closer look...

### Get bounds and mid levels for all glaciers and years
Iterate over all (profiled) mass balance records and get the different bounds for each glacier and every year. Store them in multi-dimenstional dictionaries (bounds, mid_level) with wgms id and years as primary and secondary keys, respectively.

In [ ]:
# create dictionaries as containers
bounds = dict()
mid_levels = dict()

# iterate over all selceted glaciers
for rid, wid in zip(df_links_sel.RGI60_ID, df_links_sel.WGMS_ID):
    # copy the mb records for each glacier
    df_mb_sel = df_mb_all.loc[df_mb_all.WGMS_ID == wid].copy()
    # further just use glaciers with a 'profiled' mb record
    df_mb_sel = df_mb_sel.loc[df_mb_sel.LOWER_BOUND != 9999]
    df_mb_sel = df_mb_sel.loc[df_mb_sel.UPPER_BOUND != 9999]
    # if glacier has no profiles, set flag to FALSE
    # and continue with the next glacier
    if len(df_mb_sel) == 0:
        df_links_sel.loc[df_links_sel.RGI60_ID == rid, 'HAS_PROFILE'] = False
        continue

    # create empty sets to store profile altitudes without duplicates
    bd = set() # (lower and upper) bounds
    ml = set() # mid levels
    
    # create containers
    bd_ = dict()
    ml_ = dict()
    
    # iterate over all (unique) years with profiled mb records
    for yr in df_mb_sel.YEAR.unique():
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        
        # save all unique bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.LOWER_BOUND.values] # lower bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.UPPER_BOUND.values] # upper bounds
        
        # save bounds for the current year
        tmp = np.array(list(bd))
        tmp.sort()
        bd_[str(yr)] = tmp
        
        # calculate mean between lower and upper bounds
        mids = df_mb_sel_yr.LOWER_BOUND.values*1.
        mids += df_mb_sel_yr.UPPER_BOUND.values[:len(mids)]
        mids *= 0.5
        
        # keep all unique mid levels
        for m in mids:
            ml.add(int(m))
        
        # save mid levels for the current year
        tmp = np.array(list(ml))
        tmp.sort()
        ml_[str(yr)] = tmp
        
    
    # convert, sort and store set in dictionaries
    bounds[str(wid)] = bd_
    mid_levels[str(wid)] = ml_

### Calculate step size
Now calculate the "step size" (i.e. altitude difference for all profile levels) and perform some basics statistics (median, min/max, ...) stored in a DataFrame.

In [ ]:
# empty DataFrame as container
df_stat = pd.DataFrame()

# iterate over every glacier and every year
for wid in bounds.keys():
    # subset for current glacier
    bounds_gl = bounds[wid]
    
    # empty DataFrame for current glacier
    df_stat_gl = pd.DataFrame()
    
    # iterate over all years
    for yr in bounds_gl.keys():
        
        # subset for current year
        bounds_gl_yr = bounds_gl[yr]
        # compute step size
        difs = bounds_gl_yr[1:] - bounds_gl_yr[:-1]
        
        # compute some statistical properties
        df_stat_yr = pd.DataFrame({'min': difs.min(),
                                'max': difs.max(),
                                'median': int(round(np.median(difs))),
                                'mean' : int(round(difs.mean())),
                                'std' : int(round(difs.std()))},
                                columns=list(['min', 'max', 'median', 'mean', 'std']),
                                index=[yr])


        # concat all years into one DataFrame
        if (df_stat_gl.empty):
            df_stat_gl = df_stat_yr.copy()
        else: 
            df_stat_gl = pd.concat([df_stat_gl, df_stat_yr], axis=0)
    
    # sort by years
    df_stat_gl = df_stat_gl.sort_index()
    # create multi level index with WGMS ID and year
    index = pd.MultiIndex.from_product([[wid],df_stat_gl.index.values], names=['wid', 'year'])
    df_stat_gl.index = index
    
    # concat all glaciers into one DataFrame
    if (df_stat.empty):
        df_stat = df_stat_gl.copy()
    else: 
        df_stat = pd.concat([df_stat, df_stat_gl], axis=0)
    
    df_stat = df_stat.sort_index()

Data saved in a multi indexed DataFrame for each glacier (identified by WGMS Id) and each year, sorted in ascending order.

In [ ]:
df_stat.head()

Let's take a first look at the overall outcome of the step sizes!

In [ ]:
df_stat.describe()

What are the outliers?

In [ ]:
df_stat[df_stat['min'] > 500]

The outliers with large stepsizes are STORBREEN, HOFSJOKULL (SW, E, N):

In [ ]:
df_links_sel[df_links_sel['WGMS_ID']==302]

In [ ]:
df_mb_all[df_mb_all['WGMS_ID'] == 302][df_mb_all['LOWER_BOUND'] != 9999]

In [ ]:
df_mb_all[df_mb_all['WGMS_ID'] == 3090][df_mb_all['LOWER_BOUND'] != 9999]

In [ ]:
df_mb_all[df_mb_all['WGMS_ID'] == 3088][df_mb_all['LOWER_BOUND'] != 9999]

In [ ]:
df_mb_all[df_mb_all['WGMS_ID'] == 3089][df_mb_all['LOWER_BOUND'] != 9999]

It seems like this profile covers the whole glacier. I'd suggest to check for 'single-profile' glaciers in a prior (or further) step and eliminate them!

### Excorsion: Single mb profiles - error in wgms data

Now, since we know that these lines are errors in the wgms data I'll search for all of them and report the issue...

-> Remove all measurements when there is only one measurement for that year and glacier, because then we can not use it for MB profiles

In [ ]:
# create empty container to store indices of glaciers with just one profile
single_ind = list()
# iterate over all selceted glaciers
for rid, wid in zip(df_links_sel.RGI60_ID, df_links_sel.WGMS_ID):
    # copy the mb records for each glacier
    df_mb_sel = df_mb_all.loc[df_mb_all.WGMS_ID == wid].copy()
    # further just use glaciers with a 'profiled' mb record
    df_mb_sel = df_mb_sel.loc[df_mb_sel.LOWER_BOUND != 9999]
    df_mb_sel = df_mb_sel.loc[df_mb_sel.UPPER_BOUND != 9999]
    # if glacier has no profiles, set flag to FALSE
    # and continue with the next glacier
    for yr in df_mb_sel.YEAR.unique():
        df_mb_sel_yr = df_mb_sel[df_mb_sel.YEAR == yr]
        if len(df_mb_sel_yr) == 1:
            single_ind.append([wid, yr])
            continue
        
# convert list into DataFrame
single_ind = pd.DataFrame(single_ind, columns=['WGMS_ID', 'YEAR'])

In [ ]:
df_stat.loc[str(302)]

In [ ]:
# print DataFrame
single_ind.head()

Select information for single mb-profile glaciers from look up table

In [ ]:
# create list as container
df_mb_single = list()
# iterate over all found wgms ids and years
for _,si in single_ind.iterrows():
    row_tmp = df_links_sel[df_links_sel.WGMS_ID == si.WGMS_ID]
    df_mb_single.append(row_tmp.values[0])        

# store in DataFrame
df_mb_single = pd.DataFrame(df_mb_single, columns=row_tmp.columns)

In [ ]:
# print Data Frame
df_mb_single.WGMS_ID.unique()
#for si in df_mb_single.WGMS_ID.unique():
#    print(df_mb_all[df_mb_all.WGMS_ID == si][df_mb_all[df_mb_all.WGMS_ID == si].LOWER_BOUND!=9999])

Look at the mb table for some of those glaciers with only single-measurements for one year:

In [ ]:
df_mb_all[df_mb_all.WGMS_ID == 357][df_mb_all[df_mb_all.WGMS_ID == 357].LOWER_BOUND!=9999]

In [ ]:
df_mb_all[df_mb_all.WGMS_ID == 3088][df_mb_all[df_mb_all.WGMS_ID == 3088].LOWER_BOUND!=9999]

In [ ]:
df_mb_all[df_mb_all.WGMS_ID == 302][df_mb_all[df_mb_all.WGMS_ID == 302].LOWER_BOUND!=9999]

Delete the found 'one-profile' measurements for that specific year and glacier from the statistics data set

In [ ]:
# only remove those years of the glaciers, not the entire glaciers!!!
for _,si in single_ind[::-1].iterrows():
    #midx = pd.MultiIndex(levels=['wid', 'year'],
    #                     codes=[[str(si.WGMS_ID)],[yr]])
    if (not df_stat.loc[str(si.WGMS_ID)].empty):
            print('Single mb-profile glacier wgms-id ', si.WGMS_ID, ' of year {} deleted'.format(si.YEAR))
            df_stat = df_stat.drop(index = (str(si.WGMS_ID), str(si.YEAR)))
            #df_stat.drop(index=str(si.WGMS_ID), level=yr).copy()
            #df_stat = df_stat.drop(midx).copy()
            #print('t')
    else:
        print('Glacier wgms-id ', si.WGMS_ID, ' for the year {} already deleted or not in DataFrame!'.format(si.YEAR))
        
        
#for _,si in single_ind[::-1].iterrows():
#    try:
#        if (not df_stat.loc[str(si.WGMS_ID)].empty):
#            print('Single mb-profile glacier wgms-id ', si.WGMS_ID, ' deleted')
#            df_stat = df_stat.drop(str(si.WGMS_ID)).copy()
#            print('t')
#    except:
#        print('Glacier wgms-id ', si.WGMS_ID, ' already deleted or not in DataFrame!')


### Average step size
Let's take a look at the average 'step size' over all glaciers...
The mean is probaly less meaningfull than the median, even though the IQR seems to be quiet similar

In [ ]:
# plot simple box plot
df_stat[['mean','median']].plot.box()

The average step size is around 50 meter, and more than half of the step sizes are below 100 meters.
The question is now, wheter to use 50 meters as step size and interpolate over quiet some altitude for a bunch of glaciers or use 100 meters as step size and loose some data...

In [ ]:
# calc max values of all mean and median values
max_ = max(df_stat[['mean','median']].max())
# set bin size and create bins
bin_size = 5
bins = np.arange(0,max_+bin_size, bin_size)
# plot simple histogram
df_stat[['mean','median']].plot.hist(stacked='True', bins=bins) #, normed=1)

The histogram shows that there is a substatial number of glaciers and years with a step size around 50 meters and one around 100 meters. So let's take a look at the extrem values and foremost the maxima...

### Extrem values

These show the smallest and biggest step size for every glacier and every year.

In [ ]:
# plot simple boxplot
df_stat[['min','max']].plot.box()

In [ ]:
# calc max value for mins and maxs
max_ = max(df_stat[['min', 'max']].max())
# set bin size and set bins
bin_size = 10
bins = np.arange(0,max_+bin_size, bin_size)
# print simple histogram
df_stat[['min', 'max']].plot.hist(stacked='True', bins=bins) #, normed=1)

The number of glaciers and years with step sizes over 100 is small and most are even around or below 50 meters. 
Given that the interpolation between bigger section is not a problem, but a big step size means more data loss we will use 50 meters. This can easily be changed in a second step...

### Some utility functions

#### Linear interpolation

In [ ]:
def lin_inter(f0,f1,h0,h1,h):
    return f0 + (h-h0)/(h1-h0)*(f1-f0)

#### Find closest bounds or mid levels

In [ ]:
def closest_bound(x, step_size=50):
    return int(math.ceil(x/step_size))*step_size

In [ ]:
def lower_mid(x, step_size=50):
    if type(x) == np.ndarray:
        return (x/step_size+0.5).astype(int)*step_size-step_size/2
    return int(x/step_size+0.5)*step_size-step_size/2

In [ ]:
def upper_mid(x, step_size=50):
    if type(x) == np.ndarray:
        return ((x+step_size/2-1)/step_size).astype(int)*step_size+step_size/2
    return int((x+step_size/2-1)/step_size)*step_size+step_size/2

## Actual routine to create the interpolated MB profiles

In [ ]:
# specify the step size
step_size = 50

In [ ]:
df_mb_all.loc[df_mb_all.WGMS_ID == wid]

In [ ]:
# Iterate over all glaciers
df_mb_problems = []
for rid, wid in zip(df_links_sel.RGI60_ID, df_links_sel.WGMS_ID):
    # copy the mb records for each glacier
    df_mb_sel = df_mb_all.loc[df_mb_all.WGMS_ID == wid].copy()
    # further just use glaciers with a 'profiled' mb record
    df_mb_sel = df_mb_sel.loc[df_mb_sel.LOWER_BOUND != 9999]
    df_mb_sel = df_mb_sel.loc[df_mb_sel.UPPER_BOUND != 9999]
    # if glacier has no profiles, set HAS_PROFILE flag to FALSE
    # and continue with the next glacier
    if len(df_mb_sel) == 0:
        df_links_sel.loc[df_links_sel.RGI60_ID == rid, 'HAS_PROFILE'] = False
        continue
        
    # save years in array
    years = df_mb_sel.YEAR.unique()
        
    # create empty set to store profile mid level altitudes without duplicates
    lb = set()
    bd = set()
    
    # iterate over all (unique) years with profiled mb records
    for yr in years:
        
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        if wid == 942 or wid == 299:
            # one probably false measurement with an upper / lower bound difference of more than 300m (2838 -> 3213)
            #if len(df_mb_sel_yr.loc[df_mb_sel_yr.UPPER_BOUND - df_mb_sel_yr.LOWER_BOUND<200])>= 1:
            df_mb_sel_yr = df_mb_sel_yr.loc[df_mb_sel_yr.UPPER_BOUND - df_mb_sel_yr.LOWER_BOUND<150]
            
            #print(wid, yr)
        
        
        # check if more than one profile per year
        if (len(df_mb_sel_yr) < 2):
            # delete year from list of 'profiled' years
            years = np.delete(years, np.argwhere(years==yr))
            # print user message
            print('Glacier wgms-id ', wid, ' has just one profile in ', yr)
            # continue with next year
            continue
        
        # calculate mean between lower and upper bounds
        mids_yr = df_mb_sel_yr.LOWER_BOUND.values*1.
        mids_yr += df_mb_sel_yr.UPPER_BOUND.values[:len(mids_yr)]
        mids_yr *= 0.5
        
        # save all unique bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.LOWER_BOUND.values] # lower bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.UPPER_BOUND.values] # upper bounds
        
        # keep all unique mid levels
        for m in mids_yr:
            lb.add(int(m))
            
    # continue to next glacier if lb is emty
    if not lb:
        df_links_sel.loc[df_links_sel.RGI60_ID == rid, 'HAS_PROFILE'] = False
        continue
    
    # create array with all possible mid levels (for a given step size)
    mids_tot = np.arange(upper_mid(min(lb), step_size=step_size),
                       lower_mid(max(lb), step_size=step_size)+step_size/2,
                       step_size)
    
        
    # create a DataFrame with one column for each mid level
    # index by the year of the mb recors
    prof = pd.DataFrame(np.nan, columns=mids_tot.astype(int), index=sorted(years))

    # iterate over all years
    for yr in prof.index:
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        if wid == 942 or wid == 299:
            # one probably false measurement with an upper / lower bound difference of more than 300m (2838 -> 3213)
            #if len(df_mb_sel_yr.loc[df_mb_sel_yr.UPPER_BOUND - df_mb_sel_yr.LOWER_BOUND<200])>= 1:
            if len(df_mb_sel_yr.loc[df_mb_sel_yr.UPPER_BOUND - df_mb_sel_yr.LOWER_BOUND>=150])>0:
                df_mb_problems.append(df_mb_sel_yr)
            df_mb_sel_yr = df_mb_sel_yr.loc[df_mb_sel_yr.UPPER_BOUND - df_mb_sel_yr.LOWER_BOUND<150]
        # check if more than one profile per year
        # otherwise skip this year
        if (len(df_mb_sel_yr) < 2):
            continue
            
        # calculate mean between lower and upper bounds
        mids_yr = df_mb_sel_yr.LOWER_BOUND.values*1.
        mids_yr += df_mb_sel_yr.UPPER_BOUND.values[:len(mids_yr)]
        mids_yr *= 0.5
        # create Series to store mb measurements at corresponding mid points for current year
        prof_yr = pd.Series(df_mb_sel_yr.ANNUAL_BALANCE.values, index=mids_yr)
        
        # iterate over all mid levels
        for mid in prof.columns:
            
            # mid level in DataFrame is mid level in Series
            if (mid in prof_yr.index):
                # mb value can be directly copied
                # Value ERROR
                prof.loc[yr,int(mid)] = prof_yr[mid]
                #prof.loc[yr,int(mid)] = prof_yr[int(mid)]
                # continue loop with next mid level
                continue
                
            # check if there is a lower and upper mid level
            if (any(prof_yr.index[prof_yr.index < mid]) & any(prof_yr.index[prof_yr.index > mid])):
                low = prof_yr.index[prof_yr.index < mid][-1]
                upp = prof_yr.index[prof_yr.index > mid][0]
                # linear interpolation between closest bounds
                prof.loc[yr,int(mid)] = lin_inter(prof_yr[low], prof_yr[upp], low, upp, mid)
                continue
                
            # if mid level from DataFrame is bigger or smaller than any of the mid levels
            # of the current year do nothing, since  extrapolation is not physical!
            # eventually add lower and upper bounds to interpolation...
                
    # write DataFrame to *.csv file
    prof.to_csv(os.path.join(odir, 'mb_profiles_constant_dh', 'profile_constant_dh_WGMS-{:05d}.csv'.format(wid)))
    # set the HAS_PROFILE flag to TRUE for the current glacier
    df_links_sel.loc[df_links_sel.RGI60_ID == rid, 'HAS_PROFILE'] = True

Those are the measurements that are removed because there are at the same time measurements at finer Upper-Lower bound scale

In [ ]:
df_mb_problems[0] # measurement 23514 is removed

In [ ]:
df_mb_problems[1] # measurement 23530 is removed

In [ ]:
df_mb_problems[2] # measurement 33812 is removed!

### Some plots

In [ ]:
# select all glaciers with profile
df_links_prof = df_links_sel[df_links_sel['HAS_PROFILE']]

In [ ]:
df_links_prof[df_links_prof['RGI60_ID'] == 'RGI60-11.00897']

In [ ]:
# randomly select one glacier
#wid = df_links_prof.sample(1)['WGMS_ID'].values[0]
wid = 491
df_links_prof[df_links_prof['WGMS_ID'] == wid]

In [ ]:
# read profiles
prof_unsort = pd.read_csv(os.path.join(odir, 'mb_profiles', 'profile_WGMS-{:05d}.csv'.format(wid)), index_col=0)
prof_sort = pd.read_csv(os.path.join(odir, 'mb_profiles_constant_dh', 'profile_constant_dh_WGMS-{:05d}.csv'.format(wid)), index_col=0)

In [ ]:
prof_unsort.head() 

In [ ]:
prof_sort.head()

In [ ]:
years = prof_sort.index
n_yr = len(years)
cols = 4 if n_yr > 8 else 2
rows = math.ceil(n_yr/cols)
f, axes = plt.subplots(rows, cols, sharex='col', sharey='row', figsize=[20,16])
for i,axes_ in enumerate(axes):
    for j,ax in enumerate(axes_):
        loc = len(axes_)*i+j
        if (loc < n_yr):
            ax.plot(prof_unsort.loc[years[loc]].values, list(map(int, prof_unsort.loc[years[loc]].index)), color='k', lw=3, label='unsorted')
            ax.plot(prof_sort.loc[years[loc]].values, list(map(int, prof_sort.loc[years[loc]].index)), color='r', lw=3, label='sorted')
            x_min = ax.get_xlim()
            ax.axvline(0, linestyle=':')
            ax.text(0.5,0.9, years[loc], transform=ax.transAxes, horizontalalignment='center')
        plt.legend()

### Plot MB - profiles per Latitude and Longitude

In [ ]:
# select all glaciers with profile
df_links_prof = df_links_sel[df_links_sel['HAS_PROFILE']]

In [ ]:
# create figure
fig = plt.figure(figsize=[15,15])
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# get all wgms ids
wids = df_links_prof.WGMS_ID.values
# iterate over all wgms ids
for wid in wids:
    if wid == 2660:
        continue
    # get profile from csv file
    prof = pd.read_csv(os.path.join(odir, 'mb_profiles_constant_dh', 'profile_constant_dh_WGMS-{:05d}.csv'.format(wid)), index_col=0)
    link = df_links_prof[df_links_prof.WGMS_ID == wid]
    mb_mean = prof.mean(axis=0)
    height = list(map(int, mb_mean.index.values))
    mb = mb_mean.values
    ax.plot(mb, height, label=wid)

# add zero line
ax.axvline(0, color='k', linestyle=':')
# add axis labels
ax.set_xlabel('mass balance [mm w.e.]')
ax.set_ylabel('altitude [m a.s.l.]')
# add label
ax.legend(bbox_to_anchor=(1.35, 1.02), ncol=3, title='WGMS ID')

- we don't plot Claridenfirn (wgmsid 2660) because it has extremely negative values and would make the plot difficult to read

In [ ]:
df_links_sel[df_links_sel['WGMS_ID'] == 2660]


In [ ]:

prof = pd.read_csv(os.path.join(odir, 'mb_profiles_constant_dh', 'profile_constant_dh_WGMS-{:05d}.csv'.format(2660)), index_col=0)
prof

## old stuff from Moritz
### Actual routine - First try, didn't work out quite as planned... 

IDEA: if there are more measurement points within one 'slice' of 100 meters, the interpolated mb value will be obtained by averaging the linear interpolation between the outer and inner slice...

In [ ]:
# White glacier in Canada (WGMS ID: 0)
df_links[df_links['WGMS_ID'] == 0]

In [ ]:
def closest_bound(x, step_size=100):
    return int(math.ceil(x/step_size))*step_size

In [ ]:
def closest_mid(x, step_size=100):
    mid_size = step_size/2
    return int(math.ceil(x/mid_size))*mid_size

In [ ]:
step_size = 100

# I just use the White glacier (WGMS ID 0) as test case
for rid, wid in zip(['RGI50-03.04539'], [0]):
    # copy the mb records for each glacier
    df_mb_sel = df_mb_all.loc[df_mb_all.WGMS_ID == wid].copy()
    # further just use glaciers with a 'profiled' mb record
    df_mb_sel = df_mb_sel.loc[df_mb_sel.LOWER_BOUND != 9999]
    df_mb_sel = df_mb_sel.loc[df_mb_sel.UPPER_BOUND != 9999]
    # if glacier has no profiles, set HAS_PROFILE flag to FALSE
    # and continue with the next glacier
    if len(df_mb_sel) == 0:
        df_links_sel.loc[df_links_sel.RGI_ID == rid, 'HAS_PROFILE'] = False
        continue
        
    # create empty set to store profile mid level altitudes without duplicates
    lb = set()
    bd = set()
    
    # iterate over all (unique) years with profiled mb records
    for yr in df_mb_sel.YEAR.unique():
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        # calculate mean between lower and upper bounds
        mids = df_mb_sel_yr.LOWER_BOUND.values*1.
        mids += df_mb_sel_yr.UPPER_BOUND.values[:len(mids)]
        mids *= 0.5
        
        # save all unique bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.LOWER_BOUND.values] # lower bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.UPPER_BOUND.values] # upper bounds
        
        # keep all unique mid levels
        for m in mids:
            lb.add(int(m))
            
    
    print('All altitude steps:')
    print(sorted(list(bd)))
    
    min_bd = min(bd)
    max_bd = max(bd)
    print('Min: '+str(min_bd))
    print('Max: '+str(max_bd))
    
    tmp = np.arange(closest_bound(min_bd), closest_bound(max_bd)+step_size, step_size)
    print(tmp)
    

This works, but just for this one glacier WGMS ID 0

In [ ]:
step_size = 100

# I just use the White glacier (WGMS ID 0) as test cas
for rid, wid in zip(['RGI50-03.04539'], [2665]):
    # copy the mb records for each glacier
    df_mb_sel = df_mb_all.loc[df_mb_all.WGMS_ID == wid].copy()
    # further just use glaciers with a 'profiled' mb record
    df_mb_sel = df_mb_sel.loc[df_mb_sel.LOWER_BOUND != 9999]
    df_mb_sel = df_mb_sel.loc[df_mb_sel.UPPER_BOUND != 9999]
    # if glacier has no profiles, set HAS_PROFILE flag to FALSE
    # and continue with the next glacier
    if len(df_mb_sel) == 0:
        df_links_sel.loc[df_links_sel.RGI_ID == rid, 'HAS_PROFILE'] = False
        continue
        
    # create empty set to store profile mid level altitudes without duplicates
    lb = set()
    bd = set()
    
    # iterate over all (unique) years with profiled mb records
    for yr in df_mb_sel.YEAR.unique():
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        # calculate mean between lower and upper bounds
        mids = df_mb_sel_yr.LOWER_BOUND.values*1.
        mids += df_mb_sel_yr.UPPER_BOUND.values[:len(mids)]
        mids *= 0.5
        
        # save all unique bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.LOWER_BOUND.values] # lower bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.UPPER_BOUND.values] # upper bounds
        
        # keep all unique mid levels
        for m in mids:
            lb.add(int(m))
            
    # create array with all possible lower and upper bounds (for step size)
    bounds = np.arange(closest_bound(min(bd), step_size), closest_bound(max(bd), step_size)+step_size, step_size)
    # calculate mid levels
    mids = (bounds[:-1] + bounds[1:])/2
    
    # create a DataFrame with one column for each mid level
    # index by the year of the mb recors
    prof = pd.DataFrame(np.nan, columns=mids, index=sorted(df_mb_sel.YEAR.unique()))
    
    # iterate over all years
    for yr in df_mb_sel.YEAR.unique():
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        
        # iterate over all mid levels
        for mid in prof.columns:
            # fetch levels above and below the mid level altitude on which a linear interpolation can be performed
            lower = df_mb_sel_yr[df_mb_sel_yr['LOWER_BOUND'] < mid][df_mb_sel_yr['UPPER_BOUND'] > (mid-step_size/2)]
            upper = df_mb_sel_yr[df_mb_sel_yr['UPPER_BOUND'] > mid][df_mb_sel_yr['LOWER_BOUND'] < (mid+step_size/2)]
            # number of slices found
            l_low = len(lower)
            l_up = len(upper)
            
            # if there are no lower or upper slices, skip this mid level and continue with loop
            if(l_low == 0 | l_up == 0):
                continue
            
            # check if same levels above and below mid level
            if(l_low == l_up):
                # containers for interpolated mb and distance as weight for the average
                mb = np.zeros(l_low)
                weight = np.zeros(l_low)
                # iterate over all steps
                for i in np.arange(0,l_low):
                    # get value
                    mb_low = lower.iloc[i]['ANNUAL_BALANCE']
                    bd_low = lower.iloc[i]['LOWER_BOUND']
                    mb_up = lower.iloc[(l_up-1-i)]['ANNUAL_BALANCE']
                    bd_up = lower.iloc[i]['UPPER_BOUND']
                    # interpolate linearaly
                    mb[i] = lin_inter(mb_low, mb_up, bd_low, bd_up, mid)
                    # calculate distance from mid point
                    weight[i] = step_size/2 - abs((bd_up+bd_low)/2-mid)
                    
                # average the mb inversely weighted with distance from mid level
                mb_avg = np.average(mb, weights=weight)
                
                # save in DataFrame 
                prof.loc[yr, int(mid)] = mb_avg
                
    # write DataFrame to *.csv file
    prof.to_csv(os.path.join(odir, 'profiles_mo', 'profile_WGMS-{:05d}.csv'.format(wid)))
    # set the HAS_PROFILE flag to TRUE for the current glacier
    df_links_sel.loc[df_links_sel.RGI_ID == rid, 'HAS_PROFILE'] = True

In [ ]:
def closest_bound(x, step_size=100):
    return int(round(x/step_size))*step_size

In [ ]:
def closest_mid(x, step_size=100):
    mid_size = step_size/2
    return int(round(x/mid_size))*mid_size

In [ ]:
step_size = 50

# I just use the White glacier (WGMS ID 0) as test cas
for rid, wid in zip(['RGI50-03.04539'], [2665]):
    # copy the mb records for each glacier
    df_mb_sel = df_mb_all.loc[df_mb_all.WGMS_ID == wid].copy()
    # further just use glaciers with a 'profiled' mb record
    df_mb_sel = df_mb_sel.loc[df_mb_sel.LOWER_BOUND != 9999]
    df_mb_sel = df_mb_sel.loc[df_mb_sel.UPPER_BOUND != 9999]
    # if glacier has no profiles, set HAS_PROFILE flag to FALSE
    # and continue with the next glacier
    if len(df_mb_sel) == 0:
        df_links_sel.loc[df_links_sel.RGI_ID == rid, 'HAS_PROFILE'] = False
        continue
        
    # create empty set to store profile mid level altitudes without duplicates
    lb = set()
    bd = set()
    
    # iterate over all (unique) years with profiled mb records
    for yr in df_mb_sel.YEAR.unique():
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        # calculate mean between lower and upper bounds
        mids_ = df_mb_sel_yr.LOWER_BOUND.values*1.
        mids_ += df_mb_sel_yr.UPPER_BOUND.values[:len(mids_)]
        mids_ *= 0.5
        
        # save all unique bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.LOWER_BOUND.values] # lower bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.UPPER_BOUND.values] # upper bounds
        
        # keep all unique mid levels
        for m in mids_:
            lb.add(int(m))
            
    # create array with all possible lower and upper bounds (for step size)
    bounds = np.arange(closest_bound(min(bd), step_size), closest_bound(max(bd), step_size)+step_size, step_size)
    # calculate mid levels
    mids = (bounds[:-1] + bounds[1:])/2
    
    # create a DataFrame with one column for each mid level
    # index by the year of the mb recors
    prof = pd.DataFrame(np.nan, columns=mids.astype(int), index=sorted(df_mb_sel.YEAR.unique()))
    
    # iterate over all years
    for yr in [2000]:
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        
        # iterate over all mid levels
        for mid in prof.columns:
            print(mid)
            # fetch levels above and below the mid level altitude on which a linear interpolation can be performed
            slices = df_mb_sel_yr[df_mb_sel_yr['LOWER_BOUND'] <= mid][df_mb_sel_yr['UPPER_BOUND'] >= mid]
            
            print('Slices:')
            print(slices[['LOWER_BOUND', 'UPPER_BOUND', 'ANNUAL_BALANCE']])

            # number of slices
            l_slices = len(slices)
            
            # just continue if there are slices around the mid level 
            if(l_slices != 0):
                
                # containers for interpolated mb and distance as weight for the average
                mb = np.zeros(l_slices)
                weight = np.zeros(l_slices)
                # iterate over all steps
                for i,s in enumerate(slices):
                    # get values
                    mb_ = s['ANNUAL_BALANCE']
                    bd_low = s['LOWER_BOUND']
                    bd_up = s['UPPER_BOUND']
                    # interpolate linearaly
                    mb[i] = lin_inter(mb_low, mb_up, bd_low, bd_up, mid)
                    # calculate distance from mid point
                    weight[i] = step_size/2 - abs((bd_up+bd_low)/2-mid)
                    
                # average the mb inversely weighted with distance from mid level
                if (sum(weight)==0):
                    weight = None
                mb_avg = np.average(mb, weights=weight)
                
                # save in DataFrame 
                prof.loc[yr, int(mid)] = mb_avg
                
    # write DataFrame to *.csv file
    prof.to_csv(os.path.join(odir, 'profiles_mo', 'profile_WGMS-{:05d}.csv'.format(wid)))
    # set the HAS_PROFILE flag to TRUE for the current glacier
    df_links_sel.loc[df_links_sel.RGI_ID == rid, 'HAS_PROFILE'] = True

In [ ]:
[df_mb_sel_yr['UPPER_BOUND'] < (mid+step_size/2)]

In [ ]:
bd

In [ ]:
prof

In [ ]:
sorted(list(lb))

In [ ]:
df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == 2000]
mid=150

In [ ]:
lower = df_mb_sel_yr[df_mb_sel_yr['LOWER_BOUND'] < mid][df_mb_sel_yr['UPPER_BOUND'] > (mid-step_size/2)]
print(lower)

In [ ]:
upper = df_mb_sel_yr[df_mb_sel_yr['UPPER_BOUND'] > mid][df_mb_sel_yr['LOWER_BOUND'] < (mid+step_size/2)]
print(upper)

Check for special case, for that write in a file...

In [ ]:
step_size = 100


# open file
f = open('workfile', 'w')


# iterate over all glaciers
for rid, wid in zip(df_links_sel.RGI_ID, df_links_sel.WGMS_ID):
    # copy the mb records for each glacier
    df_mb_sel = df_mb_all.loc[df_mb_all.WGMS_ID == wid].copy()
    # further just use glaciers with a 'profiled' mb record
    df_mb_sel = df_mb_sel.loc[df_mb_sel.LOWER_BOUND != 9999]
    df_mb_sel = df_mb_sel.loc[df_mb_sel.UPPER_BOUND != 9999]
    # if glacier has no profiles, set HAS_PROFILE flag to FALSE
    # and continue with the next glacier
    if len(df_mb_sel) == 0:
        df_links_sel.loc[df_links_sel.RGI_ID == rid, 'HAS_PROFILE'] = False
        continue
        
    # create empty set to store profile mid level altitudes without duplicates
    lb = set()
    bd = set()
    
    f.write('Glacier: '+str(wid)+'\n')
        
    
    # iterate over all (unique) years with profiled mb records
    for yr in df_mb_sel.YEAR.unique():
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        # calculate mean between lower and upper bounds
        mids = df_mb_sel_yr.LOWER_BOUND.values*1.
        mids += df_mb_sel_yr.UPPER_BOUND.values[:len(mids)]
        mids *= 0.5
        
        # save all unique bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.LOWER_BOUND.values] # lower bounds
        [bd.add(int(b)) for b in df_mb_sel_yr.UPPER_BOUND.values] # upper bounds
        
        # keep all unique mid levels
        for m in mids:
            lb.add(int(m))
            
    # create array with all possible lower and upper bounds (for step size)
    bounds = np.arange(closest_bound(min_bd, step_size), closest_bound(max_bd, step_size)+step_size, step_size)
    # calculate mid levels
    mids = (bounds[:-1] + bounds[1:])/2
    
    # create a DataFrame with one column for each mid level
    # index by the year of the mb recors
    prof = pd.DataFrame(np.nan, columns=mids, index=sorted(df_mb_sel.YEAR.unique()))
    
    # iterate over all years
    for yr in df_mb_sel.YEAR.unique():
        # subset dataframe for each year
        df_mb_sel_yr = df_mb_sel.loc[df_mb_sel.YEAR == yr]
        
        f.write('Year: '+str(yr)+'\n')
        
        
        # iterate over all mid levels
        for mid in prof.columns:
            
            f.write('Mid: '+str(mid)+'\t')
            
            # fetch levels above and below the mid level altitude on which a linear interpolation can be performed
            lower = df_mb_sel_yr[df_mb_sel_yr['LOWER_BOUND'] < mid][df_mb_sel_yr['UPPER_BOUND'] > (mid-step_size/2)]
            upper = df_mb_sel_yr[df_mb_sel_yr['UPPER_BOUND'] > mid][df_mb_sel_yr['LOWER_BOUND'] < (mid+step_size/2)]
            # number of slices found
            l_low = len(lower)
            l_up = len(upper)
            
            # if there are no lower or upper slices, skip this mid level and continue with loop
            if(l_low == 0 | l_up == 0):
                f.write('No lower or upper slices\n')
                continue
            
            # check if same levels above and below mid level
            if(l_low == l_up):
                f.write('Same number of upper and lower slices\n')
            elif (l_low > l_up):
                f.write('More lower than upper slices\n')
            else: # l_low < l_up
                f.write(r'More upper than lower slices\n')
                
        
        f.write('\n')
    
    f.write('--------------------------\n')
    
    
# close file
f.close()

In [ ]:
def closest_mid(x, step_size=100):
    mid_size = step_size/2
    return int(math.ceil(x/mid_size))*mid_size

In [ ]:
closest_mid()

### Links: add some stats

In [ ]:
# Handle various RGI versions
df_links_sel.rename(columns = {'RGI_ID':'RGI50_ID'}, inplace = True)
df_links_sel['RGI40_ID'] = df_links_sel['RGI50_ID']
df_links_sel['RGI40_ID'] = [rid.replace('RGI50', 'RGI40') for rid in df_links_sel['RGI40_ID']]

In [ ]:
# Get the RGI
import geopandas as gpd
import glob, os
frgi = '/home/mowglie/Documents/rgi50_allglaciers.csv'
if not os.path.exists(frgi):
    # one time action only
    fs = list(sorted(glob.glob("/home/mowglie/disk/Data/GIS/SHAPES/RGI/RGI_V5/*/*_rgi50_*.shp")))[2:]
    out = []
    for f in fs:
        sh = gpd.read_file(f).set_index('RGIId')
        del sh['geometry']
        del sh['GLIMSId']
        del sh['Name']
        out.append(sh)
    mdf = pd.concat(out)
    mdf.to_csv(frgi)
mdf = pd.read_csv(frgi, index_col=0, converters={'GlacType': str, 'RGIFlag':str, 'BgnDate':str, 
                                                 'O1Region': str, 'O2Region':str})
mdf['RGI_REG'] = [rid.split('-')[1].split('.')[0] for rid in mdf.index]
# add region names
sr = gpd.read_file('/home/mowglie/disk/Data/GIS/SHAPES/RGI/RGI_V5/00_rgi50_regions/00_rgi50_O1Regions.shp')
sr = sr.drop_duplicates('Secondary_').set_index('Secondary_')[['Primary_ID']]
sr['Primary_ID'] = [i + ': ' + s for i, s in sr.Primary_ID.iteritems()]
mdf['RGI_REG_NAME'] = sr.loc[mdf.RGI_REG].Primary_ID.values

In [ ]:
# Read glacier attrs
key1 = {'0': 'Glacier',
        '1': 'Ice cap',
        '2': 'Perennial snowfield',
        '3': 'Seasonal snowfield',
        '9': 'Not assigned',
        }

key2 = {'0': 'Land-terminating',
        '1': 'Marine-terminating',
        '2': 'Lake-terminating',
        '3': 'Dry calving',
        '4': 'Regenerated',
        '5': 'Shelf-terminating',
        '9': 'Not assigned',
        }

def is_tidewater(ttype):
    return 

mdf['GlacierType'] = [key1[gtype[0]] for gtype in mdf.GlacType]
mdf['TerminusType'] = [key2[gtype[1]] for gtype in mdf.GlacType]
mdf['IsTidewater'] = [ttype in ['Marine-terminating', 'Lake-terminating'] for ttype in mdf.TerminusType]

In [ ]:
# add lons and lats and other attrs to the WGMS ones
smdf = mdf.loc[df_links_sel.RGI50_ID]
df_links_sel['CenLon'] = smdf.CenLon.values
df_links_sel['CenLat'] = smdf.CenLat.values
df_links_sel['GlacierType'] = smdf.GlacierType.values
df_links_sel['TerminusType'] = smdf.TerminusType.values
df_links_sel['IsTidewater'] = smdf.IsTidewater.values
df_links_sel['RGI_REG_NAME'] = smdf.RGI_REG_NAME.values

In [ ]:
df_links_sel = df_links_sel[['CenLon', 'CenLat',
                             'POLITICAL_UNIT', 'NAME', 'WGMS_ID', 'PSFG_ID', 'WGI_ID', 'GLIMS_ID',
                             'RGI40_ID', 'RGI50_ID', 'RGI_REG', 'RGI_REG_NAME', 'GlacierType', 'TerminusType', 
                             'IsTidewater', 'N_MB_YRS', 'HAS_PROFILE', 'REMARKS']]
df_links_sel.to_csv(os.path.join(odir, 'rgi_wgms_links_20170217_RGIV5.csv'.format(wid)), index=False)

## Some plots 

In [ ]:
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')
pdir = '/home/mowglie/Documents/git/fmaussion.github.io/images/blog/wgms-links'

In [ ]:
df_links_sel['N_MB_YRS'].plot(kind='hist', color='C2', bins=np.arange(21)*5);
plt.xlim(5, 100);
plt.ylabel('Number of glaciers')
plt.xlabel('Length of the timeseries (years)');
plt.tight_layout();
plt.savefig(os.path.join(pdir, 'nglacier-hist.png'), dpi=150)

In [ ]:
import cartopy
import cartopy.crs as ccrs

f = plt.figure(figsize=(12, 7))
ax = plt.axes(projection=ccrs.Robinson())
# mark a known place to help us geo-locate ourselves
ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
ax.stock_img()
ax.add_feature(cartopy.feature.COASTLINE);
s = df_links_sel.loc[df_links_sel.N_MB_YRS < 10]
print(len(s))
ax.scatter(s.CenLon, s.CenLat, label='< 10 MB years', s=50,
           edgecolor='k', facecolor='C0', transform=ccrs.PlateCarree(), zorder=99)
s = df_links_sel.loc[(df_links_sel.N_MB_YRS >= 10) & (df_links_sel.N_MB_YRS < 30)]
print(len(s))
ax.scatter(s.CenLon, s.CenLat, label='$\geq$ 10 and < 30 MB years', s=50,
           edgecolor='k', facecolor='C1', transform=ccrs.PlateCarree(), zorder=99)
s = df_links_sel.loc[df_links_sel.N_MB_YRS >= 30]
print(len(s))
ax.scatter(s.CenLon, s.CenLat, label='$\geq$ 30 MB years', s=50,
           edgecolor='k', facecolor='C2', transform=ccrs.PlateCarree(), zorder=99)
plt.title('WGMS glaciers with at least 5 years of mass-balance data')
plt.legend(loc=4, frameon=True)
plt.tight_layout();
plt.savefig(os.path.join(pdir, 'glacier-map.png'), dpi=150)

In [ ]:
df_links_sel.TerminusType.value_counts().to_frame()

In [ ]:
ax = sns.countplot(x='RGI_REG', hue="TerminusType", data=df_links_sel);

In [ ]:
md = pd.concat([mdf.GlacierType.value_counts().to_frame(name='RGI V5').T, 
          df_links_sel.GlacierType.value_counts().to_frame(name='WGMS').T]
          ).T
md

In [ ]:
md = pd.concat([mdf.TerminusType.value_counts().to_frame(name='RGI V5').T, 
          df_links_sel.TerminusType.value_counts().to_frame(name='WGMS').T]
          ).T
md

In [ ]:
area_per_reg = mdf[['Area', 'RGI_REG_NAME']].groupby('RGI_REG_NAME').sum()
area_per_reg['N_WGMS'] = df_links_sel.RGI_REG_NAME.value_counts()
area_per_reg = area_per_reg.reset_index()

In [ ]:
sns.barplot(x="Area", y="RGI_REG_NAME", data=area_per_reg);

In [ ]:
area_per_reg['N_WGMS_PER_UNIT'] = area_per_reg.N_WGMS / area_per_reg.Area * 1000

In [ ]:
sns.barplot(x="N_WGMS", y="RGI_REG_NAME", data=area_per_reg);  # , palette=sns.husl_palette(19, s=.7, l=.5)
plt.ylabel('')
plt.xlabel('')
plt.title('Number of WGMS glaciers per RGI region');
plt.tight_layout();
plt.savefig(os.path.join(pdir, 'barplot-ng.png'), dpi=150)

In [ ]:
sns.barplot(x="N_WGMS_PER_UNIT", y="RGI_REG_NAME", data=area_per_reg);
plt.ylabel('')
plt.xlabel('')
plt.title('Number of WGMS glaciers per 1,000 km$^2$ of ice, per RGI region');
plt.tight_layout();
plt.savefig(os.path.join(pdir, 'barplot-perice.png'), dpi=150)

In [ ]:
nmb_yrs = df_links_sel[["RGI_REG", 'N_MB_YRS']].groupby("RGI_REG").sum()
i = []
for k, d in nmb_yrs.iterrows():
     i.extend([k] * d.values[0])
df = pd.DataFrame()
df["RGI_REG"] = i
ax = sns.countplot(x="RGI_REG", data=df)